<a href="https://colab.research.google.com/github/intelligent-environments-lab/occupant_centric_grid_interactive_buildings_course/blob/main/src/notebooks/tutorials/introduction_to_energy_plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to EnergyPlus
---

[EnergyPlus](https://energyplus.net) is a physics-based whole building energy performance simulation software. It is widely used in academia and academia by building energy professionals for design optimization, retrofit analyses, energy audits, and performance verification to mention a few.

One way to use EnergyPlus is to build an energy model from scratch. However, this approach requires a significant expertise and time investment as well as access to data that may not be readily available. The other route which we adopt in this class is to take advantage of stock energy models that have been developed and vetted by experts in the energy research field. To this end, we will make use of the [End-Use Load Profiles (EULP) for the U.S. Building Stock](https://www.nrel.gov/buildings/end-use-load-profiles.html) dataset that has about 900 thousand energy models representing the residential and commercial building stock in the United States. The dataset has had a few releases over the years however we will make use of the `resstock-amy2018-2021-release-1`. This [dataset](https://data.openei.org/s3_viewer?bucket=oedi-data-lake&prefix=nrel-pds-building-stock%2Fend-use-load-profiles-for-us-building-stock%2F2021%2Fresstock_amy2018_release_1%2F) is
the 2021 release of about 500,000 energy models and their simulation results for a 2018
actual meteorological year weather.

In this tutorial, you will learn how to install EnergyPlus, run a simulation on an EULP energy model, and visualize its results using a _Pythonic_ interface.

## Installing EnergyPlus
---

There are two ways you could go about installing EnergyPlus. The first is to install it locally in your machine and always have it when you need it. Note that if you take this approach, you will also have to run this notebook locally as Colab is unable to point to your local file structure.

The second way, and recommended for this class, is to install it temporarily in the Colab environment runtime. This approach is local-machine-agnostic and will help avoid typically installation problems that are specific to your local machine.

Whichever approach you choose, make sure you install the `v9.6.0` version of EnergyPlus as it is what will be compatible with the dataset we use.

To install to local computer, [download the installer for your specific platform](https://github.com/NREL/EnergyPlus/releases/tag/v9.6.0).

To install to the current Colab environment runtime, uncomment and execute the next cell:

In [ ]:
# download installation script
!rm -f /content/EnergyPlus-9.6.0-f420c06a69-Linux-Ubuntu20.04-x86_64.sh
!wget https://github.com/NREL/EnergyPlus/releases/download/v9.6.0/EnergyPlus-9.6.0-f420c06a69-Linux-Ubuntu20.04-x86_64.sh

# install
!chmod +x /content/EnergyPlus-9.6.0-f420c06a69-Linux-Ubuntu20.04-x86_64.sh
!echo -e "yes\n" | sudo /content/EnergyPlus-9.6.0-f420c06a69-Linux-Ubuntu20.04-x86_64.sh

# to capture C-level stdout/stderr pipes in Python
!pip install wurlitzer

# check EP
print('\nEnergyPlus Version:')
!energyplus -version

# delete installer
!rm -f /content/EnergyPlus-9.6.0-f420c06a69-Linux-Ubuntu20.04-x86_64.sh

Confirm that the last printout above reports that `EnergyPlus Version 9.6.0` was installed.

## Interfacing with EULP dataset
---

We will test our installation by simulating a random single-family attached building in Fulton County (Atlanta), Goergia (like our smart neighborhood building). We have written a Python wrapper called `DOE_XStock` around the EULP dataset that allows use to easily query its metadata and run simulations. First we will install this wrapper and other relevant dependencies:

In [ ]:
# wrapper for EULP dataset
!pip install git+https://github.com/intelligent-environments-lab/DOE_XStock.git@v1-develop

# for plotting figures
!pip install matplotlib

### How to retrieve building metadata in EULP dataset

With the wrapper installed we can now interface with the dataset. The models in the EULP dataset are referenced using building IDs. Let us return the metadata of all single-family attached building in Fulton County. We begin by making the necessary imports:

In [ ]:
import os
from doe_xstock.end_use_load_profiles import EndUseLoadProfiles
import matplotlib.pyplot as plt
import pandas as pd

Next we initialize the dataset:

In [ ]:
eulp = EndUseLoadProfiles(
    dataset_type='resstock',
    weather_data='amy2018',
    year_of_publication=2021,
    release=1,
)

We can now return the metadata:

In [ ]:
metadata = eulp.metadata.metadata.get(filters={
    'in.resstock_county_id': ['GA, Fulton County'],
    'in.geometry_building_type_recs': ['Single-Family Attached'],
})

with pd.option_context('display.max_columns', None):
    display(metadata.head())
    print('Number of represented single-family attached buildings in GA, Fulton County:', metadata.shape[0])

From the printout, there are 116 buildings that match our filter criteria. We will now randomly pick a building to simulate:

In [ ]:
BLDG_ID = metadata.sample(1, random_state=0).iloc[0].name
print('Picked building ID:', BLDG_ID)

We can also retrieve a dictionary containing all the metadata for this particular building:

In [ ]:
print(BLDG_ID, 'metadata:')
display(eulp.get_building(BLDG_ID).metadata)

## Simulating an EULP building energy model
---

Simulating energy models is quite straightforward using the `DOE_XStock` wrapper. However, there is one variable, a filpath, you need to retrieve called the `EnergyPlus IDD`. `IDD` stands for `Input Data Dictionary` and is an ASCII (text) file containing a list of all possible EnergyPlus objects and a specification of the data each object requires. It is EnergyPlus version specific and helps the internal model editor in `DOE_XStock` aware of the _version language_.

If you installed EnergyPlus within Colab then, the path defined in the next cell points to the correct `IDD`.

For local installations, it will be in a similar path as below but the root will be specific to your machine. Mac users should be able to find the EnergyPlus directory in their Applications directory. Once found, replace the `IDD_FILEPATH` below with the correct path:

In [ ]:
IDD_FILEPATH = '/usr/local/EnergyPlus-9-6-0/PreProcess/IDFVersionUpdater/V9-6-0-Energy+.idd'

if os.path.isfile(IDD_FILEPATH):
    print('IDD exists!')
else:
    print('Provided IDD does not exist:', IDD_FILEPATH)

To then simulate the building is as simple as parsing its ID to the relevant function with the `IDD_FILEPATH` we have defined:

In [ ]:
SIMULATION_ID = f'simulation-{BLDG_ID}'
building = eulp.simulate_building(
    BLDG_ID, 
    IDD_FILEPATH, 
    simulation_id=SIMULATION_ID, 
    output_directory=SIMULATION_ID
)

## Analyzing and visualizing results from simulation
---

With our simulation completed, all that is left is to look at how the building performed with its as provided parameters seen in its metadata printout. EnergyPlus exports results in a `.sqlite` file as well as `.csv` file. It is up to you which output you want to work with. The `SQLite` database is easier to use when trying to manage results from many variables and thermal zones in a building. The `CSV` text file is a more familiar interface for most people. Let us start by using the `.csv` to retrieve and plot the outdoor and indoor dry-bulb temperature as well as the air system cooling and heating rates in any of the zones of the building. We will use the `SQLite` approach in a later tutorial.

In [ ]:
results = building.simulator.get_result_csv()

with pd.option_context('display.max_columns', None):
    display(results)

From inspecting the dataframe, we can find the four relevant columns. Also notice that by default, the simulation is on a 15-minute time step and is done for a full year?

In [ ]:
columns = [
    "Environment:Site Outdoor Air Drybulb Temperature [C](TimeStep)",
    "LIVING ZONE:Zone Air Temperature [C](TimeStep)",
    "LIVING ZONE:Zone Air System Sensible Heating Rate [W](TimeStep)",
    "LIVING ZONE:Zone Air System Sensible Cooling Rate [W](TimeStep)",
]

Sometimes, you may not know what variables are availble or reported from the simulation. There are two output files that can help with this. They are:

1. *.rdd (Report variable Data Dictionary): list of output variables available from the run.
2. *.mdd (report Meter Data Ditionary): list of output meters available from the run.

They are in the same path as the simulation results. You can locate the path by executing the following:

In [ ]:
print('Simulation output directory:', building.simulator.output_directory)

Those two files have a long list of variables that may be intimidating if you have no idea what they reference or come from. The easiest way to learn about them is either through a Google search or directly search the [EnergyPlus documentation](https://bigladdersoftware.com/epx/docs/9-6/input-output-reference/).

Now let us go back to visualizing our results:

In [ ]:
for c in columns:
    fig, ax = plt.subplots(1, 1, figsize=(10, 2.0))
    x = results.index
    y = results[c]
    ax.plot(x, y)
    ax.set_xlabel('Time step')
    ax.set_title(c)
    plt.show()

Do you spot anything interesting from the figures? You can now go back to simulate a different building if you wish and perhaps compare its results to this.

## Conclusion
---

We have learned how to install EnergyPlus and how to interface with the End-Use Load Profiles (EULP) for the U.S. Building Stock as well as run simulations on its energy models. This tutorial was short but prepares you for longer tutorials where you will run a number of simulations.

It is encouraged that you take some time to read through the references and further reading links. Most importantly, practice, practice, practice!

## Further reading and references
---

1. [Introduction to building energy performance simulation](https://github.com/intelligent-environments-lab/occupant_centric_grid_interactive_buildings_course/blob/main/src/lectures/CE397_OCCGEB_Sp24_05.pdf)
4. [End-Use Load Profiles (EULP) for the U.S. Building Stock](https://www.nrel.gov/buildings/end-use-load-profiles.html)
2. [EnergyPlus quickstart](https://energyplus.net/quick-start)
2. [EnergyPlus](https://energyplus.net)
2. [EnergyPlus documentation](https://bigladdersoftware.com/epx/docs/9-6/input-output-reference/)
3. [EULP DOE_XStock wrapper documentation](https://github.com/intelligent-environments-lab/DOE_XStock/tree/v1-develop)
5. [Eppy tutorial for manipulating EnergyPlus IDF file](https://eppy.readthedocs.io/en/latest/Main_Tutorial.html)